In [1]:
'''
Adapated from: https://github.com/CVxTz/kinship_prediction/blob/master/code/vgg_face.py
to parse the images and data
This is NOT used to make the model!
Additionally, this is run on a Windows OS. Some code may not be need if it is on a UNIX or LINUX OS.
'''
import pandas as pd
from glob import glob #for finding files recursively
from collections import defaultdict

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.utils
import torchvision.datasets as dset

from torch import optim
from torch.utils.data import DataLoader,Dataset
from torchvision.models import *
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
from PIL import Image

In [2]:
!unzip '/content/train.zip' -d '/content/train'

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/F0601/MID2/P12074_face1.jpg  
  inflating: /content/train/F0601/MID2/P11969_face4.jpg  
  inflating: /content/train/F0601/MID2/P11883_face1.jpg  
  inflating: /content/train/F0601/MID2/P11900_face1.jpg  
  inflating: /content/train/F0601/MID2/P06303_face1.jpg  
  inflating: /content/train/F0601/MID2/P11885_face1.jpg  
  inflating: /content/train/F0601/MID2/P11875_face1.jpg  
  inflating: /content/train/F0601/MID2/P06300_face7.jpg  
  inflating: /content/train/F0601/MID2/P11878_face1.jpg  
  inflating: /content/train/F0601/MID2/P11898_face1.jpg  
  inflating: /content/train/F0601/MID2/P06394_face4.jpg  
  inflating: /content/train/F0601/MID2/P11879_face1.jpg  
  inflating: /content/train/F0601/MID2/P11877_face1.jpg  
  inflating: /content/train/F0601/MID2/P06344_face2.jpg  
  inflating: /content/train/F0601/MID2/P06279_face1.jpg  
  inflating: /content/train/F0601/MID2/P06288_face1.jpg  
  inflating: /content

In [3]:
relationshipsCSV = "../content/train_relationships.csv"
train_images_folder = "../content/train/"
#what is the difference between train and train-face?
val_set = "F09" #can change this, this is randomly generated

'''
Idea of hypertuning:
using different validation sets for each iteration
maybe instead of using only only family as the validation, we can use a group
or change the validation set for each iteration of training the model
'''


'\nIdea of hypertuning:\nusing different validation sets for each iteration \nmaybe instead of using only only family as the validation, we can use a group\nor change the validation set for each iteration of training the model \n'

In [18]:
availble_images = glob(train_images_folder + "*/*/*.jpg")

'''
Only for windows pc:
'''
for a in range(0, len(availble_images)):
    availble_images[a] = availble_images[a].replace("\\", "/")

all_ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in availble_images] #all the people

['../content/train/F0845/MID3/P08930_face2.jpg',
 '../content/train/F0845/MID3/P08928_face2.jpg',
 '../content/train/F0845/MID3/P08927_face2.jpg',
 '../content/train/F0845/MID3/P08925_face2.jpg',
 '../content/train/F0845/MID3/P08924_face2.jpg',
 '../content/train/F0845/MID3/P08931_face2.jpg',
 '../content/train/F0845/MID1/P08925_face1.jpg',
 '../content/train/F0845/MID1/P08926_face2.jpg',
 '../content/train/F0845/MID1/P08923_face1.jpg',
 '../content/train/F0845/MID1/P08927_face1.jpg',
 '../content/train/F0845/MID1/P08929_face1.jpg',
 '../content/train/F0845/MID1/P08931_face1.jpg',
 '../content/train/F0845/MID1/P08930_face1.jpg',
 '../content/train/F0845/MID1/P08928_face1.jpg',
 '../content/train/F0845/MID1/P08922_face1.jpg',
 '../content/train/F0845/MID1/P08924_face1.jpg',
 '../content/train/F0845/MID2/P08930_face3.jpg',
 '../content/train/F0845/MID2/P08929_face2.jpg',
 '../content/train/F0845/MID2/P08926_face1.jpg',
 '../content/train/F0845/MID2/P08923_face2.jpg',
 '../content/train/F

In [30]:
#creating the training set
train_images = [x for x in availble_images if val_set not in x]
train_person_to_images_map = defaultdict(list)
train_not_relationship_person_to_images_map = defaultdict(list)
for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)


#creating the validation set
val_images = [x for x in availble_images if val_set in x]
val_person_to_images_map = defaultdict(list)
for x in val_images:
     val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

train_person_to_images_map

defaultdict(list,
            {'F0845/MID3': ['../content/train/F0845/MID3/P08930_face2.jpg',
              '../content/train/F0845/MID3/P08928_face2.jpg',
              '../content/train/F0845/MID3/P08927_face2.jpg',
              '../content/train/F0845/MID3/P08925_face2.jpg',
              '../content/train/F0845/MID3/P08924_face2.jpg',
              '../content/train/F0845/MID3/P08931_face2.jpg'],
             'F0845/MID1': ['../content/train/F0845/MID1/P08925_face1.jpg',
              '../content/train/F0845/MID1/P08926_face2.jpg',
              '../content/train/F0845/MID1/P08923_face1.jpg',
              '../content/train/F0845/MID1/P08927_face1.jpg',
              '../content/train/F0845/MID1/P08929_face1.jpg',
              '../content/train/F0845/MID1/P08931_face1.jpg',
              '../content/train/F0845/MID1/P08930_face1.jpg',
              '../content/train/F0845/MID1/P08928_face1.jpg',
              '../content/train/F0845/MID1/P08922_face1.jpg',
              '../conte

In [54]:
#read from the csv to create a list of tuples
relationships = pd.read_csv(relationshipsCSV)



relationship_pairs = [(row['p1'], row['p2']) for index, row in relationships.iterrows()] # Create a list of tuples
relationship_pairs = [x for x in relationship_pairs if x[0] in all_ppl and x[1] in all_ppl] #data cleaning
train = [x+(1,) for x in relationship_pairs if val_set not in x[0]] #label as related
val = [x+(1,) for x in relationship_pairs if val_set in x[0]] #label as related

relationshipDict = defaultdict(list)

for item in relationship_pairs:
  relationshipDict[item[0]].append(item[1])


#create more data by finding the 'negative pair'
notRelationshipDict = defaultdict(list)

for item in all_ppl[3:10]:
  relationshipExists =  relationshipDict[item]
  #find the first person
  if len(relationshipExists)>0:
    for item2 in all_ppl:
      if item2 not in relationshipExists and item2 not in notRelationshipDict[item]:
        notRelationshipDict[item].append(item2)
        break


#convert both dictionaries into a list of tuples with 1, 0 as labels
notRelationshipDict

defaultdict(list,
            {'F0845/MID1': ['F0845/MID1',
              'F0845/MID2',
              'F0367/MID4',
              'F0367/MID3']})

In [8]:
'''
Adapted from: https://github.com/ale-mauro/Kinship-Recognition/blob/main/KinshipRecognition_Naive_DenseNet161_ResNet.ipynb
Obtaining the training and validation set to use the following model:
https://keras.io/examples/vision/siamese_network/

The idea is to split to obtain a "triple" set:
1. an anchor
2. positive
3. negative
'''

'\nAdapted from: https://github.com/ale-mauro/Kinship-Recognition/blob/main/KinshipRecognition_Naive_DenseNet161_ResNet.ipynb\nObtaining the training and validation set to use the following model:\nhttps://keras.io/examples/vision/siamese_network/\n\nThe idea is to split to obtain a "triple" set:\n1. an anchor\n2. positive\n3. negative\n'

In [9]:
import random


#use the pytorch built in Dataset function
class trainingDataset(Dataset):
    def __init__(self,imageFolderDataset, relationships, transform=None):
        self.imageFolderDataset = imageFolderDataset
        self.relationships = relationships #choose either train or val dataset to use
        self.transform = transform


    def __getitem__(self,index):

        img0_info = self.relationships[index][0] #gets the first item in the tuple

        img0_path = glob("train/"+img0_info+"/*.jpg") #gets the file path
        img0_path = random.choice(img0_path) #chose randomically a photo of the individual

        # Found all candidates related to person in img0
        candidate_relationship = [x for x in self.relationships if x[0]==img0_info or x[1]==img0_info]

        # Randomly choose whether to use a positive (1) or a negative (0) example
        if candidate_relationship==[]: # in this case, this should never happen.
            choose_positive_example = 0
        else:
            # Choose randomicaly to have positive or negative example: 1 means related, and 0 means non-related.
            choose_positive_example = random.randint(0,1)

        # If we have to choose positive example, we get the second person from related relationship
        if choose_positive_example==1:
            img1_info = random.choice(candidate_relationship)#choose the second person from related relationships
            # remember that candidate_relationship = (img0_info, 'Fxxx/MIDx') OR ('Fxxx/MIDx', img0_info)),
            # so img1 is the element of the tuple that is not img0_info
            if img1_info[0] != img0_info:
                img1_info = img1_info[0]
            else:
                img1_info=img1_info[1]

            #randomly choose a img of second person
            img1_path = glob("train/"+img1_info+"/*.jpg")
            img1_path = random.choice(img1_path)

        # In case we have to choose negative sample, we get the second person randomicaly (0 means non-related)
        else:
            randChoose = True #in case the random chosen person is related to first person
            while randChoose:
                img1_path = random.choice(self.imageFolderDataset.imgs)[0]
                img1_info = img1_path.split("/")[-3] + "/" + img1_path.split("/")[-2]
                randChoose = False
                #if we (unforunatly) choose a person who is related to the first one, randomly choose another person
                for x in candidate_relationship:
                    if x[0]==img1_info or x[1]==img1_info:
                        randChoose = True
                        break

        # Now we have two images (they are related or not)
        img0 = Image.open(img0_path)
        img1 = Image.open(img1_path)

        # Transform images
        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)

        # The returned data from dataloader is img=[batch_size,channels,width,length], should_get_same_class=[batch_size,label]
        return img0, img1 , choose_positive_example

    def __len__(self):
        return len(self.relationships)#essential for choose the num of data in one epoch


In [14]:
folder_dataset = dset.ImageFolder(root='train/')
BATCH_SIZE=64
IMG_SIZE=100

#Training set and training loader
trainset = trainingDataset(imageFolderDataset=folder_dataset,
                                        relationships=train,
                                        transform=transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)),
                                                                      transforms.ToTensor()
                                                                      ]))


len(trainset)

3066

In [11]:
'''
More models:
https://viso.ai/computer-vision/deepface/
'''

'\nMore models:\nhttps://viso.ai/computer-vision/deepface/ \n'